## 6.2 循环神经网络

### 6.2.1 不含隐藏状态的神经网络

> 1. **给定样本数为n,特征数为d的小批量数据样本  $\boldsymbol{X} \in \mathbb{R}^{n \times d}$,设隐藏层的激活函数$\phi$,则隐藏层的输出$\boldsymbol{H} \in \mathbb{R}^{n \times d}$;**
$$\boldsymbol{H}=\phi(\boldsymbol{X}\boldsymbol{W}_{xh} + \boldsymbol{b}_{h})$$
**其中隐藏层权重参数$\boldsymbol{W}_{xh} \in \mathbb{R}^{n \times d}$; 隐藏层偏差参数$\boldsymbol{b}_{h} \in \mathbb{R}^{1 \times h}$**

> 2. **把隐层变量$\boldsymbol{H}$作为输出层的输入,且输出个数各位为q(如分类),则输出层的输出为**
$$\boldsymbol{O}=\boldsymbol{H}\boldsymbol{W}_{hq} + \boldsymbol{b}_{h}$$
**其中$\boldsymbol{O} \in \mathbb{R}^{n \times q}$;权重参数$\boldsymbol{W}_{hq} \in \mathbb{R}^{h \times q}$;输出层偏差参数$\boldsymbol{b}_q \in \mathbb{R}^{1 \times q}$;如果分类,可通过`softmax`函数**